<a href="https://colab.research.google.com/github/anilbhatt1/EVA4P1_S15_MaskRCNN/blob/master/EVA4P1_S15_DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from datetime import datetime 
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-05-09 08:21:23.352026


In [23]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
import PIL
import numpy as np
from PIL import Image, ImageOps
import zipfile
import sys

In [0]:
'''
----------------------------------
|    (x_a,y_a)       (x_b,y_b)   |
|    *---------------*           |
|    |               |           |
|    |               |           |
|    *---------------*           |
|    (x_c,y_c)       (x_d,y_d)   |
|                                |
----------------------------------
'''

In [0]:
def pos_generator(bg_size,fg_size,positions):
    lst  = []
    b_width, b_height = bg_size
    f_width, f_height = fg_size
    max_y = b_height - f_height
    max_x = b_width - f_width
    print(max_y,max_x)
    for i in range(positions):
        x = np.random.randint(low=0, high=max_x, size=1)[0]
        y = np.random.randint(low=0, high=max_y, size=1)[0]
        x_a, y_a = x,y 
        x_b, y_b = x + f_width, y
        x_c, y_c = x, y + f_height
        x_d, y_d = x + f_width, y + f_height
        tup = (x,y,b_width,b_height,f_width,f_height,x_a, y_a, x_b, y_b, x_c, y_c, x_d, y_d) 
        lst.append(tup)
    return lst     

In [0]:
def fg_bg_creation(pos_list):
    global i, log_fg_bg, log_fg_bg_mask

    for tup in range(int(positions/2)):
        i += 1
        print('i,positions:',tup ,pos_list[tup])        
        x,y = pos_list[tup][0:2]       
        background = Image.open(f'{path_bg}{bg_img}') 
        foreground = Image.open(f'{path_fg}{fg_img}')                                       
        background.paste(foreground, (x, y), foreground)
        fg_bg_img_name  = 'Img_fg_bg_' + str(i) + '.jpg'
        background.save(f'{path_fg_bg}{fg_bg_img_name}')
        string = f'{path_fg_bg}{fg_bg_img_name},{bg_img},{fg_img},' + str(pos_list[tup][2:]) + '\n'
        log_fg_bg.write(string)
                
        foreground = foreground.convert('L')
        foreground = foreground.point(lambda x: 255 if x > 0 else 0 , '1')
        background = Image.open(f'{path_bg}{bg_img}')
        background = background.convert('L')
        background = background.point(lambda x: 0 if x > 0 else 255 , '1')
        background.paste(foreground, (x, y), foreground)
        fg_bg_mask_name  = 'Img_fg_bg_mask' + str(i) + '.jpg'
        background.save(f'{path_fg_bg_mask}{fg_bg_mask_name}')
        string = f'{path_fg_bg_mask}{fg_bg_mask_name},{bg_img},{fg_img},' + str(pos_list[tup][2:]) + '\n'
        log_fg_bg_mask.write(string) 

    for tup in range(int(positions/2),int(positions)):
        i += 1
        print('i,positions:',tup ,pos_list[tup])         
        x,y = pos_list[tup][0:2]           
        background  = Image.open(f'{path_bg}{bg_img}')  
        background  = ImageOps.mirror(background)
        foreground  = Image.open(f'{path_fg}{fg_img}')           
        background.paste(foreground, (x, y), foreground)                  
        fg_bg_img_name = 'Img_fg_bg_' + str(i) + '.jpg'
        background.save(f'{path_fg_bg}{fg_bg_img_name}') 
        string = f'{path_fg_bg}{fg_bg_img_name},{bg_img},{fg_img},' + str(pos_list[tup][2:]) + '\n'  
        log_fg_bg.write(string)

        foreground = foreground.convert('L')
        foreground = foreground.point(lambda x: 255 if x > 0 else 0 , '1')
        background = Image.open(f'{path_bg}{bg_img}')
        background = ImageOps.mirror(background)
        background = background.convert('L')
        background = background.point(lambda x: 0 if x > 0 else 255 , '1')        
        background.paste(foreground, (x, y), foreground)
        fg_bg_mask_name  = 'Img_fg_bg_mask' + str(i) + '.jpg'
        background.save(f'{path_fg_bg_mask}{fg_bg_mask_name}')
        string = f'{path_fg_bg_mask}{fg_bg_mask_name},{bg_img},{fg_img},' + str(pos_list[tup][2:]) + '\n'
        log_fg_bg_mask.write(string)                         

In [32]:
path_bg         = '/content/gdrive/My Drive/Test_S15/Test_BG_Images/'
path_fg         = '/content/gdrive/My Drive/Test_S15/Test_FG_Images/'
path_fg_bg      = '/content/sample_data/FG_BG/'
path_fg_bg_mask = '/content/sample_data/FG_BG_Mask/'
log_fg_bg       = open('/content/sample_data/FG_BG_Filename_Logs.txt', "w")
log_fg_bg_mask  = open('/content/sample_data/FG_BG_Mask_Filename_Logs.txt', "w")
positions  = 40
i          = 0

print('Image Creation - Start Time:',datetime.now())

for bg_img in os.listdir(path_bg):
    if bg_img.endswith('jpg'):
       for fg_img in os.listdir(path_fg):
           if fg_img.endswith('png'):
              bg = Image.open(f'{path_bg}{bg_img}') 
              fg = Image.open(f'{path_fg}{fg_img}') 
              pos_list   = pos_generator(bg.size, fg.size, positions)
              fg_bg_creation(pos_list)
    if i%50000 == 0:
       print('Progress:',datetime.now(),'Images:',i)

print('Total Images saved:',i)
log_fg_bg.close()
log_fg_bg_mask.close()

Image Creation - Start Time: 2020-05-09 08:30:50.185248
97 114
i,positions: 0 (53, 45, 192, 192, 78, 95, 53, 45, 131, 45, 53, 140, 131, 140)
i,positions: 1 (46, 50, 192, 192, 78, 95, 46, 50, 124, 50, 46, 145, 124, 145)
i,positions: 2 (91, 27, 192, 192, 78, 95, 91, 27, 169, 27, 91, 122, 169, 122)
i,positions: 3 (14, 20, 192, 192, 78, 95, 14, 20, 92, 20, 14, 115, 92, 115)
i,positions: 4 (87, 25, 192, 192, 78, 95, 87, 25, 165, 25, 87, 120, 165, 120)
i,positions: 5 (96, 68, 192, 192, 78, 95, 96, 68, 174, 68, 96, 163, 174, 163)
i,positions: 6 (75, 36, 192, 192, 78, 95, 75, 36, 153, 36, 75, 131, 153, 131)
i,positions: 7 (26, 58, 192, 192, 78, 95, 26, 58, 104, 58, 26, 153, 104, 153)
i,positions: 8 (27, 47, 192, 192, 78, 95, 27, 47, 105, 47, 27, 142, 105, 142)
i,positions: 9 (93, 94, 192, 192, 78, 95, 93, 94, 171, 94, 93, 189, 171, 189)
i,positions: 10 (72, 40, 192, 192, 78, 95, 72, 40, 150, 40, 72, 135, 150, 135)
i,positions: 11 (55, 63, 192, 192, 78, 95, 55, 63, 133, 63, 55, 158, 133, 158)
i

In [0]:
zipname_fg_bg      = 'FG_BG'
zipname_fg_bg_mask = 'FG_BG_Mask'

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

In [0]:
zipfolder(zipname_fg_bg, '/content/sample_data/')
zipfolder(zipname_fg_bg_mask, '/content/sample_data/')

In [0]:
!cp FG_BG.zip '/content/gdrive/My Drive/Test_S15/Test_FG_BG/'
!cp FG_BG_Mask.zip '/content/gdrive/My Drive/Test_S15/Test_FG_BG/'

In [0]:
#drive.flush_and_unmount()